<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [67]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [68]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-09 08:14:22--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-09 08:14:22 (117 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [69]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-09 08:14:22--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.2.112, 172.253.63.128, 142.251.16.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.2.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.003s  

2022-09-09 08:14:22 (168 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [70]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [71]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [72]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [73]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [74]:
len(train_df),len(test_df)

(7613, 3263)

In [75]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:0 (not a disaster)
Text
 I get this feeling that society will collapse or implode. So don't be a hero and play your part. 

---

target:0 (not a disaster)
Text
 Various issues fail to derail homes bid http://t.co/zhsLl7swBh 

---

target:1 (real disaster)
Text
 airplane crashes on house in Colombia 12 people die in accident https://t.co/ZhJlfLBHZL 

---

target:1 (real disaster)
Text
 Today is the day Hiroshima got Atomic bomb 70 years ago.  - The 'sanitised narrative' of Hiroshima's atomic bombing http://t.co/GKpANz7vg0 

---

target:0 (not a disaster)
Text
 @Deeeznvtzzz bring the body bags tho 

---



## splitting the data into training and validation sets

In [76]:
from sklearn.model_selection import train_test_split

In [77]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [78]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [79]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [80]:
max_vocab_length = 10000
max_length = 15

In [81]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [82]:
text_vectorizer.adapt(train_sentences)

In [83]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [84]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 @marc_holl @NenniCook @AitchKayCee @vixstuart @benjbeckwith It's not pretty ??#disaster #GBBO #mudslide       
 Vectorized version:
 [[   1    1    1 6990    1   37   34  766   75  896  456    0    0    0
     0]]


In [85]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [86]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [87]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Myanmar floods: Childfund https://t.co/pQHQ4JnZTT
 and International Needs https://t.co/FX0W2Sq05F and CARE Aust @CAREemergencies appeals         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01855372, -0.04523927,  0.02467824, ...,  0.00405605,
          0.00573482,  0.03981965],
        [-0.00156206,  0.03300122, -0.0386345 , ..., -0.0164157 ,
          0.03399045, -0.03001713],
        [-0.01066814,  0.02498093, -0.0234879 , ..., -0.03145998,
          0.00943094, -0.01068077],
        ...,
        [-0.01066814,  0.02498093, -0.0234879 , ..., -0.03145998,
          0.00943094, -0.01068077],
        [ 0.02906567,  0.0398321 ,  0.00686115, ..., -0.03612822,
         -0.04303981, -0.02841567],
        [ 0.02906567,  0.0398321 ,  0.00686115, ..., -0.03612822,
         -0.04303981, -0.02841567]]], dtype=float32)>

In [88]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

('Myanmar floods: Childfund https://t.co/pQHQ4JnZTT\n and International Needs https://t.co/FX0W2Sq05F and CARE Aust @CAREemergencies appeals',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01855372, -0.04523927,  0.02467824, -0.01430274, -0.02493899,
        -0.00411435,  0.00700141, -0.03350115, -0.03423893, -0.0327791 ,
        -0.02745627,  0.00660857, -0.02279561,  0.02977772,  0.04651665,
        -0.01810521,  0.01130497,  0.04511159,  0.03270679, -0.03435986,
         0.04455068,  0.01388418, -0.04741208, -0.04385288, -0.02325903,
        -0.03013489,  0.02507674,  0.02507338, -0.0391105 ,  0.04501433,
        -0.02236831,  0.0402762 ,  0.04195671,  0.02661257,  0.00564956,
         0.01034846,  0.02312441,  0.04916947, -0.00800375, -0.01811805,
        -0.01880453,  0.02992792, -0.04844686, -0.02062073, -0.01247429,
        -0.03122522,  0.02614347, -0.03477388, -0.00840826,  0.02063111,
        -0.00779017,  0.03554443, -0.02581874,  0.04695902,  0.00073962,
     

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [90]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [91]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [92]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [93]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [94]:
from helper_functions import create_tensorboard_callback

In [95]:
SAVE_DIR = 'model_logs'

In [96]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [97]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [98]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220909-081423
Epoch 1/5
215/215 [==============================] - 2s 7ms/step - loss: 0.6124 - accuracy: 0.6888 - val_loss: 0.5367 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.4413 - accuracy: 0.8180 - val_loss: 0.4697 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3463 - accuracy: 0.8610 - val_loss: 0.4573 - val_accuracy: 0.7979
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2833 - accuracy: 0.8920 - val_loss: 0.4640 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2366 - accuracy: 0.9107 - val_loss: 0.4813 - val_accuracy: 0.7822


In [99]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4813 - accuracy: 0.7822


[0.4812912344932556, 0.7821522355079651]

In [100]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.38043258],
       [0.75887614],
       [0.997875  ],
       [0.14308414],
       [0.10355353],
       [0.9457128 ],
       [0.916288  ],
       [0.9926376 ],
       [0.9708717 ],
       [0.31774992]], dtype=float32)

In [101]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [102]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7821522309711286,
 'precision': 0.7856661242905698,
 'recall': 0.7821522309711286,
 'f1': 0.7795215466562155}

In [106]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([False, False, False, False])